In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import joblib
from tensorflow.keras.models import load_model
import numpy as np
import datetime
from tensorflow.keras.losses import MeanSquaredError

# Load model and scaler
model = load_model('Predict_Energy_Consumption_model_1.6.h5', custom_objects={'mse': MeanSquaredError()})
scaler = joblib.load('scaler.pkl')

def generate_features(date):
    # Generate features based on the provided date
    year = date.year
    semester = 1 if date.month <= 6 else 2
    quarter = (date.month - 1) // 3 + 1
    week_in_year = date.isocalendar()[1]
    day_in_year = date.timetuple().tm_yday
    month = date.month
    day_in_week = date.weekday()  # 0=Monday, ..., 6=Sunday

    # Create one-hot encoding for the day of the week
    day_in_week_features = [False] * 7
    day_in_week_features[day_in_week] = True

    return [
        year, semester, quarter, week_in_year, day_in_year, month,
        *day_in_week_features  # Spread day-in-week features
    ]

def predict_consumption():
    try:
        date_str = date_entry.get()
        date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
        
        # Generate features
        features = generate_features(date)
        
        # Scale features
        scaled_features = scaler.transform([features])
        
        # Predict
        prediction = model.predict(scaled_features)
        result_label.config(text=f"Predicted Power Consumption: {prediction[0][0]:.2f} kWh")
    except Exception as e:
        messagebox.showerror("Error", str(e))

def analyze_month():
    try:
        date_str = month_entry.get()
        date = datetime.datetime.strptime(date_str, "%Y-%m")
        month_start = datetime.datetime(date.year, date.month, 1)
        days_in_month = (datetime.datetime(date.year + (date.month // 12), (date.month % 12) + 1, 1) - month_start).days
        
        daily_predictions = []
        for day in range(1, days_in_month + 1):
            current_date = month_start + datetime.timedelta(days=day - 1)
            features = generate_features(current_date)
            scaled_features = scaler.transform([features])
            prediction = model.predict(scaled_features)
            daily_predictions.append(prediction[0][0])
        
        total_consumption = sum(daily_predictions)
        average_consumption = total_consumption / days_in_month
        result_label.config(text=f"Total: {total_consumption:.2f} kWh, Avg: {average_consumption:.2f} kWh/day")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# GUI setup
root = tk.Tk()
root.title("Power Consumption Predictor")
root.geometry("500x300")
root.configure(bg="#f7f7f7")

style = ttk.Style()
style.configure("TLabel", font=("Arial", 12), background="#f7f7f7")
style.configure("TButton", font=("Arial", 12), padding=5)
style.configure("TEntry", font=("Arial", 12))

header_label = ttk.Label(root, text="Power Consumption Predictor", font=("Arial", 16, "bold"), background="#f7f7f7")
header_label.grid(row=0, column=0, columnspan=3, pady=10)

# Date prediction section
date_label = ttk.Label(root, text="Enter Date (YYYY-MM-DD):")
date_label.grid(row=1, column=0, padx=10, pady=5, sticky="e")

date_entry = ttk.Entry(root, width=20)
date_entry.grid(row=1, column=1, padx=10, pady=5)

predict_button = ttk.Button(root, text="Predict", command=predict_consumption)
predict_button.grid(row=1, column=2, padx=10, pady=5)

# Monthly analysis section
month_label = ttk.Label(root, text="Enter Month (YYYY-MM):")
month_label.grid(row=2, column=0, padx=10, pady=5, sticky="e")

month_entry = ttk.Entry(root, width=20)
month_entry.grid(row=2, column=1, padx=10, pady=5)

analyze_button = ttk.Button(root, text="Analyze Month", command=analyze_month)
analyze_button.grid(row=2, column=2, padx=10, pady=5)

# Result display section
result_label = ttk.Label(root, text="", font=("Arial", 12, "bold"), foreground="#333")
result_label.grid(row=3, column=0, columnspan=3, pady=20)

# Run the application
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


C:\Users\sasin\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
